## Notebook: Calibration of the Heston Model  

This notebook focuses on pricing a **1-year call option** using the **Heston model** with the following parameters:  

- $v_0 = \sigma^2 = 0.05$  
- $\kappa = 0.5$  
- $\eta = 0.05$  
- $\theta = 0.2$  
- $\rho = -0.75$  
- $S_0 = 100$  
- $q = 0.01$ (dividend yield)  
- $r = 0.05$ (risk-free rate)  

### **Strike Prices**  
We consider three different strike prices:  
$$K = 90, 100, 110$$  

### **Pricing Methods**  
The option price is computed using the following methods:  
1. **Fast Fourier Transform (FFT)** with Simpson’s rule *(see Example Session 2)*.  
2. **Monte Carlo simulation** using the **Euler scheme**.  
3. **Monte Carlo simulation** using the **Milstein scheme**.  


In [3]:
# imports 
from  tiny_pricing_utils import *
from scipy.interpolate import interp1d
import numpy as np

In [4]:
# Market data
S0 = 100
q = 0.01
r = 0.05
sigma = 0.25
K = [90, 100, 110]
T = 1

In [5]:
# Hetson variables !!eta 
V0 = 0.05
kappa = 0.5 
eta = 0.05
theta = 0.2
rho = -0.75

In [6]:
# variables for FFT pricing (eta != eta_cm)
eta_cm = 0.25
N = 4096
alpha = 1.5
lambda_ = (2 * np.pi) / (N* eta_cm)
b = (N * lambda_) / 2
print(lambda_)
print(b)

0.006135923151542565
12.566370614359172


In [12]:
# Define a finer grid from the original strikes to interpolate on (from min to max log strike)
log_strikes = np.arange(-b, b, lambda_)

# Grid for the Fourier transform variable
v = np.arange(0, N * eta_cm, eta_cm) 
u = v - (alpha + 1) * 1j

# Define rho (g of x) using the characteristic function evaluated at (v - (alpha + 1) * i)
rho_cm = np.exp(-r * T) * cf_Heston(u, S0, r, q, V0, kappa, eta, theta, rho, T) / \
    (alpha**2 + alpha - v**2 + 1j * (2 * alpha + 1) * v)

# Simpson's Rule Coefficients
simpson_1 = 1/3  # First coefficient
simpson = (3 + (-1)**np.arange(2, N+1)) / 3  # Alternating coefficients starting from index 2
simpson_int = np.concatenate(([simpson_1], simpson))  # Combine with the first coefficient

# Apply Simpson's Rule correction in the FFT computation
fft_result_simpson = np.fft.fft(np.exp(1j * v * b) * rho_cm * eta_cm * simpson_int)

# Extract real part of the result
a_simpson = np.real(fft_result_simpson)

# Calculate call option prices using the Carr-Madan formula
calls_simpson = (1 / np.pi) * np.exp(-alpha * log_strikes) * a_simpson

# Reconstruct strike prices from log-strikes
KK = np.exp(log_strikes)

# Compute call prices using FFT pricing formula for both rules
calls_simpson = (1 / np.pi) * np.exp(-alpha * log_strikes) * a_simpson

# Define specific strike prices for interpolation
specific_strikes = np.array([90, 100, 110])

# Create cubic spline interpolation functions
spline_simpson = interp1d(KK, calls_simpson, kind='cubic', fill_value="extrapolate")

# Get interpolated option prices for specific strikes
interpolated_simpson = spline_simpson(specific_strikes)

# Print results
print(f"Interpolated Call Option Prices for K = {specific_strikes} using Simpson's Rule: \n {interpolated_simpson}")


Interpolated Call Option Prices for K = [ 90 100 110] using Simpson's Rule: 
 [16.93747985 10.6163149   5.86707143]
